In [1]:
# Estudo paradas tratadas
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
from shapely.geometry import box, Point, Polygon, MultiPolygon
from shapely.wkt import loads
import folium as fl
import pandas as pd

In [2]:
dt = pd.read_csv("../../../databases/luiz_dataset/paradas_linestrings_ida.csv", converters={'parada': pd.eval})
linhas = pd.read_csv('../../../databases/linhas/Linhas.csv')

In [3]:
dt

,Unnamed: 0,linha,parada
0,0,0.018,"[[-15.747487243955707, -47.89600866323914, 351..."
1,1,0.946,"[[-15.80778852625573, -47.96642705712294, 2256..."
2,2,0.373,"[[-15.866555379423424, -47.96504713913905, 605..."
3,3,503.300,"[[-15.638937404577504, -47.82260762728512, 491..."
4,4,531.100,"[[-15.638937404577504, -47.82260762728512, 491..."
...,...,...,...
482,482,0.844,"[[-15.884075085352835, -48.129561044093, 4228]..."
483,483,0.172,"[[-15.866555379423424, -47.96504713913905, 605..."
484,484,805.600,"[[-15.867119301987762, -48.02312295895697, 534..."
485,485,506.200,"[[-15.638004360182045, -47.821280315734576, 48..."


In [4]:
# Faz o teste para ver se tem uma linha no seu dataframe
if '0.205' in [str(dado) for dado in list(dt.linha.unique())]:
    print('Tem')
else:
    print('Não tem')

Tem


In [5]:
# Fazendo o plot do 813
dt_813 = dt[dt['linha']==0.205]

In [6]:
dt_813.parada

186    [[-16.003489982135445, -48.053122011503056, 19...
Name: parada, dtype: object

In [7]:
# Salvando a rota em um array de tuplas
from pyproj import Transformer
#transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326", always_xy=True)

for ponto in dt_813.parada:
    m = fl.Map(location=[-15.76050, -47.78134], zoom_start=10)
    print(ponto)
    for lat, long, _ in ponto:
        #transformed_coords = transformer.transform(lat, long)
        fl.Circle([lat, long], fill_color='#FF0000', radius=15).add_to(m)
m  

[[-16.003489982135445, -48.053122011503056, '1900'], [-15.998391002544192, -48.05290170811915, '1901'], [-15.925732999999996, -48.046239, '7913'], [-15.92573532555408, -48.0462197207585, '3815'], [-16.0280877634412, -48.06458066149397, '1972'], [-16.025931495067827, -48.06560316078725, '1991'], [-16.02389074482397, -48.061564500079214, '1992'], [-16.024412167253924, -48.06301687416831, '1993'], [-16.025856969993338, -48.06591711994143, '1994'], [-16.0343415161678, -48.06209332768591, '2012'], [-16.03416282671877, -48.062037651276945, '2013'], [-16.029834929993385, -48.06177955994188, '1971'], [-16.032138486006442, -48.06576889093318, '2010'], [-16.03322396811534, -48.063873073461096, '2011'], [-16.03313272640074, -48.063758134519254, '2014'], [-16.03161633999342, -48.06616474642197, '2015'], [-16.029058927886254, -48.07027202811057, '2016'], [-16.024609490803094, -48.06955590078616, '1930'], [-16.023697359190226, -48.072352919371134, '1963'], [-16.02447046514923, -48.07105199332125, '1

### Preparação para filtragem de paradas que não fazem parte de uma rota. 

O projeto consiste em calcular a média das distâncias entre as paradas e em seguida colocar um limiar abaixo para que as paradas muito próximas uma das outras sejam eliminadas.

$$ média\ das\ distâncias = \sum_{i=0}^{n} \frac{distâncias_{i}}{n-1}$$

$n: número\ de\ paradas$

As etapas de dessa filtragem são:

**Passo 1: Preparação dos Dados**

- Deixar o dataframe das paradas em ordem ao longo da rota e indexadas.
- Calular as distâncias entre paradas consecutivas.

**Passo 2: Definição de Limiares**

- Calcular a média das distâncias entre as paradas.
- Calcular o desvio padrão das distâncias entre as paradas.
- Definir um limiar que seja uma combinação da média e do desvio padrão, por exemplo, `limiar = média + k * desvio_padrao`, onde `k` é um fator que você pode ajustar com base na sensibilidade desejada.

**Passo 3: Identificação e Eliminação de Paradas Próximas**

- Iniciar um loop para percorrer todas as paradas da rota, começando pela primeira.
- Para cada parada atual, calcular a distância até a próxima parada.
- Se a distância for menor que o limiar definido no Passo 2, eliminar a próxima parada (atualize os dados originais para remover essa parada).
- Reordenar a lista de paradas, se necessário, após a eliminação.
- Voltar ao passo 3º caso não tenha chegado até a ultima parada.

**Passo 4: Repetição e Verificação**

- Repetir o Passo 3 até que não seja mais possível eliminar paradas (ou seja, a distância entre todas as paradas seja maior que o limiar).
- Isso garantirá que todas as paradas próximas demais sejam removidas.
- Verificar no gráfico se a qualidade da representação da rota melhorou visualizando os resultados no gráfico.


In [8]:
from shapely.geometry import Point
import geopandas as gpd

# Etapa 1: Calcular a distância entre dois pontos da linestring de forma simples
p1 = Point(ponto[0][0], ponto[0][1])
p2 = Point(ponto[1][0], ponto[1][1])

# Verifica se os pontos estão corretos
print("O ponto 1 eh:", p1)
print("O ponto 2 eh:", p2)

O ponto 1 eh: POINT (-16.003489982135445 -48.053122011503056)
O ponto 2 eh: POINT (-15.998391002544192 -48.05290170811915)


In [9]:
# Criar um GeoDataFrame com os pontos
data = {'Name': ['Point 1', 'Point 2'], 'geometry': [p1, p2]}
gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
gdf


,Name,geometry
0,Point 1,POINT (-16.00349 -48.05312)
1,Point 2,POINT (-15.99839 -48.05290)


In [10]:
# Calcular a distância entre os dois pontos
distance = gdf.geometry.iloc[0].distance(gdf.geometry.iloc[1])

print("Distância entre os pontos:", distance)
print(type(distance))


Distância entre os pontos: 0.005103736518764752
<class 'float'>


In [11]:
# Genralizando para apenas um única linha
# Função para cálculo de distâncias entre dois pontos
def calcDistance(p1, p2) -> float:
    distance = p1.distance(p2)
    return distance

143